In [171]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

In [172]:
trash = pd.read_csv('../data/trash_hauler_report_with_lat_lng.csv')

In [173]:
trash.head()

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y,LONGITUDE,LATITUDE
0,25270,11/1/2017,Trash - Backdoor,"house with the wheel chair ramp, they share dr...",3817 Crouch Dr,37207.0,RED RIVER,3205,2.0,1727970.412,686779.4781,-86.815392,36.217292
1,25274,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,37218.0,RED RIVER,4202,1.0,1721259.366,685444.7996,-86.838103,36.213470
2,25276,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 Thunderbird Dr,37209.0,RED RIVER,4205,20.0,1707026.753,659887.4716,-86.885562,36.142923
3,25307,11/1/2017,Trash - Curbside/Alley Missed Pickup,missed,2603 old matthews rd,37207.0,WASTE IND,2206,2.0,1735691.771,685027.2459,-86.789170,36.212652
4,25312,11/1/2017,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,37209.0,RED RIVER,4203,20.0,1710185.772,664205.1011,-86.874995,36.154861


## Q: Determine total amount of damages due to missed pickups.

In [174]:
trash['Incident Address'] = trash['Incident Address'].str.upper()
trash['Trash Hauler'] = trash['Trash Hauler'].str.upper()
trash.head()

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y,LONGITUDE,LATITUDE
0,25270,11/1/2017,Trash - Backdoor,"house with the wheel chair ramp, they share dr...",3817 CROUCH DR,37207.0,RED RIVER,3205,2.0,1727970.412,686779.4781,-86.815392,36.217292
1,25274,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 CLARKSVILLE PIKE,37218.0,RED RIVER,4202,1.0,1721259.366,685444.7996,-86.838103,36.213470
2,25276,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 THUNDERBIRD DR,37209.0,RED RIVER,4205,20.0,1707026.753,659887.4716,-86.885562,36.142923
3,25307,11/1/2017,Trash - Curbside/Alley Missed Pickup,missed,2603 OLD MATTHEWS RD,37207.0,WASTE IND,2206,2.0,1735691.771,685027.2459,-86.789170,36.212652
4,25312,11/1/2017,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 CROLEY DR,37209.0,RED RIVER,4203,20.0,1710185.772,664205.1011,-86.874995,36.154861


In [175]:
trash['Request'].unique()

array(['Trash - Backdoor', 'Trash - Curbside/Alley Missed Pickup',
       'Trash Collection Complaint', 'Damage to Property'], dtype=object)

In [176]:
missed_pickup = trash['Request'] == 'Trash - Curbside/Alley Missed Pickup'
trash_missed = trash[missed_pickup]
display(trash_missed)

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y,LONGITUDE,LATITUDE
1,25274,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 CLARKSVILLE PIKE,37218.0,RED RIVER,4202,1.0,1721259.366,685444.7996,-86.838103,36.213470
2,25276,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 THUNDERBIRD DR,37209.0,RED RIVER,4205,20.0,1707026.753,659887.4716,-86.885562,36.142923
3,25307,11/1/2017,Trash - Curbside/Alley Missed Pickup,missed,2603 OLD MATTHEWS RD,37207.0,WASTE IND,2206,2.0,1735691.771,685027.2459,-86.789170,36.212652
4,25312,11/1/2017,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 CROLEY DR,37209.0,RED RIVER,4203,20.0,1710185.772,664205.1011,-86.874995,36.154861
8,25330,11/1/2017,Trash - Curbside/Alley Missed Pickup,Missed.,4484 LAVERGNE COUCHVILLE PIKE,37013.0,RED RIVER,4210,33.0,1794533.514,618749.3427,-86.588364,36.031728
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20221,267125,11/1/2019,Trash - Curbside/Alley Missed Pickup,MISSED...NEIGHBORS MISSED,2731 MURFREESBORO PIKE,37013.0,RED RIVER,4502,32.0,1781137.263,632448.5511,-86.633970,36.069130
20222,267126,11/1/2019,Trash - Curbside/Alley Missed Pickup,entire alley,"1621 LONG AVE, NASHVILLE, TN 37206, UNITED STATES",37206.0,METRO,9508,6.0,1749711.399,669201.6016,-86.741242,36.169482
20223,267130,11/1/2019,Trash - Curbside/Alley Missed Pickup,missed several,"2943 WINDEMERE CIR, NASHVILLE, TN 37214, UNITE...",37214.0,RED RIVER,1502,15.0,1770293.388,674936.3038,-86.671647,36.185643
20224,267134,11/1/2019,Trash - Curbside/Alley Missed Pickup,Caller stated trash was missed & were only pic...,"3325 MURFREESBORO PIKE, NASHVILLE, TN 37013, U...",37013.0,RED RIVER,4502,32.0,1785224.998,627146.4002,-86.620025,36.054637


In [177]:
duplicates = trash_missed.duplicated(subset = ['Incident Address'], keep = 'first')
fines = trash_missed[duplicates]

In [178]:
fines.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3820 entries, 46 to 20224
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Request Number    3820 non-null   int64  
 1   Date Opened       3820 non-null   object 
 2   Request           3820 non-null   object 
 3   Description       3815 non-null   object 
 4   Incident Address  3813 non-null   object 
 5   Zip Code          3809 non-null   float64
 6   Trash Hauler      3562 non-null   object 
 7   Trash Route       3553 non-null   object 
 8   Council District  3810 non-null   float64
 9   State Plan X      3811 non-null   float64
 10  State Plan Y      3811 non-null   float64
 11  LONGITUDE         3811 non-null   float64
 12  LATITUDE          3811 non-null   float64
dtypes: float64(6), int64(1), object(6)
memory usage: 417.8+ KB


In [179]:
empty_address = fines[fines['Incident Address'].isnull()]
display(empty_address)

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y,LONGITUDE,LATITUDE
875,35857,12/29/2017,Trash - Curbside/Alley Missed Pickup,daughters car parked in front,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1154,39689,1/17/2018,Trash - Curbside/Alley Missed Pickup,Trash pick up not done for Tuesday 1/16/18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1346,41604,1/24/2018,Trash - Curbside/Alley Missed Pickup,cart still out,NaN,37218.0,RED RIVER,3203,1.0,1715186.450,682289.9617,-86.858594,36.204659
2794,58920,4/6/2018,Trash - Curbside/Alley Missed Pickup,missed- trash,NaN,37206.0,METRO,9503,5.0,1747402.049,674741.0561,-86.749210,36.184650
2839,59517,4/10/2018,Trash - Curbside/Alley Missed Pickup,Trash was not pick up last week.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3121,62051,4/22/2018,Trash - Curbside/Alley Missed Pickup,They forgot to pick up trash from apartment co...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11049,157096,3/20/2019,Trash - Curbside/Alley Missed Pickup,Trash has not been picked up on the whole street.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [180]:
fines_cleaned = fines.dropna(subset = ['Incident Address'])
fines_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3813 entries, 46 to 20224
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Request Number    3813 non-null   int64  
 1   Date Opened       3813 non-null   object 
 2   Request           3813 non-null   object 
 3   Description       3808 non-null   object 
 4   Incident Address  3813 non-null   object 
 5   Zip Code          3807 non-null   float64
 6   Trash Hauler      3560 non-null   object 
 7   Trash Route       3551 non-null   object 
 8   Council District  3808 non-null   float64
 9   State Plan X      3809 non-null   float64
 10  State Plan Y      3809 non-null   float64
 11  LONGITUDE         3809 non-null   float64
 12  LATITUDE          3809 non-null   float64
dtypes: float64(6), int64(1), object(6)
memory usage: 417.0+ KB


In [181]:
total_damages = fines_cleaned['Incident Address'].count() * 200
print(total_damages)

762600


> <div class="alert alert-block alert-info">
    > A: Total Damages = $762,600

## Q: What other types of complaints are there?

In [182]:
complaint = trash['Request'] == 'Trash Collection Complaint'
trash_complaints = trash[complaint]
trash_complaints.head(20)

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y,LONGITUDE,LATITUDE
5,25317,11/1/2017,Trash Collection Complaint,left trash cart in middle of driveway instead ...,3602 FLORAL DR,37211.0,RED RIVER,4304,16.0,1751660.164,643215.2011,-86.733980,36.098140
7,25327,11/1/2017,Trash Collection Complaint,"Trash out on time, miss again Tuesday. ALLEY",1816 JO JOHNSTON AVE,37203.0,METRO,9208,21.0,1731459.367,666013.6012,-86.802988,36.160330
11,25350,11/1/2017,Trash Collection Complaint,Missed- 4th week in a row.,3210 WHITES CREEK PIKE,37207.0,RED RIVER,3201,3.0,1726432.966,691919.2003,-86.820750,36.231374
13,25366,11/1/2017,Trash Collection Complaint,trash truck took cart \r\n3 guys on truck did ...,2200 OSAGE ST,37208.0,METRO,9201,21.0,1728644.757,672074.0991,-86.812692,36.176913
16,25393,11/1/2017,Trash Collection Complaint,trash carts all tossed in road after pick up,2720 RIVERBEND DR,37214.0,RED RIVER,1501,15.0,1762455.924,693225.6657,-86.698647,36.235731
17,25407,11/1/2017,Trash Collection Complaint,Curb/Trash miss everyone already gotten pick u...,4209 LONE OAK RD,37215.0,RED RIVER,3305,25.0,1729355.438,644389.9207,-86.809509,36.100884
20,25465,11/1/2017,Trash Collection Complaint,A resident is complaining that a trash truck r...,1414 OLD HICKORY BLVD,37207.0,NaN,NaN,3.0,1738736.963,711231.7998,-86.779556,36.284699
27,25508,11/1/2017,Trash Collection Complaint,missed. Neighbor missed as well-say she is at ...,2812 27TH AVE S,37212.0,RED RIVER,2305,18.0,1729022.334,652985.6791,-86.810877,36.124488
33,25519,11/1/2017,Trash Collection Complaint,"5th time in as many weeks, Metro has not picke...",3702 HOBBS RD,37215.0,RED RIVER,3304,34.0,1721317.066,645218.7019,-86.836741,36.102974
35,25525,11/1/2017,Trash Collection Complaint,Missed my trash AGAIN.\r\ncheck your records. ...,162 ANTIOCH PIKE,37211.0,RED RIVER,4304,16.0,1753144.965,644275.2022,-86.728981,36.101082


In [183]:
trash_complaints.tail(20)

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y,LONGITUDE,LATITUDE
20092,265839,10/30/2019,Trash Collection Complaint,trash hasn't been collected for two weeks. wou...,"TREVINO PL, NASHVILLE, TN 37013, UNITED STATES",37013.0,RED RIVER,3505,29.0,1785939.670,636626.0759,-86.617810,36.080690
20096,265871,10/30/2019,Trash Collection Complaint,today makes 3 weeks missed trash/curb {sit's o...,"4001 NEBRASKA AVE, NASHVILLE, TN 37209, UNITED...",37209.0,WASTE IND,2308,24.0,1722556.201,659684.0006,-86.832964,36.142738
20099,265901,10/30/2019,Trash Collection Complaint,constantly missed for backdoor pickup. would l...,"638 JAMES AVE, NASHVILLE, TN 37209, UNITED STATES",37209.0,RED RIVER,4203,20.0,1710211.779,666316.5683,-86.874971,36.160662
20104,265963,10/30/2019,Trash Collection Complaint,paving blocking alley for trash pickup,"400 HYDE PARK, NASHVILLE, TENNESSEE, 37215",37215.0,RED RIVER,3304S,34.0,1723745.001,646234.3816,-86.828552,36.105821
20115,266137,10/30/2019,Trash Collection Complaint,"2 weeks missed Wednesday, sit's on Nevada Ave ...","301 33RD AVE N, NASHVILLE, TN 37209, UNITED ST...",37209.0,WASTE IND,2308,24.0,1725181.801,662212.8000,-86.824144,36.149746
20117,266144,10/30/2019,Trash Collection Complaint,My trash bin (and others on my street) are fre...,"1334 BURTON VALLEY RD, NASHVILLE, TN 37215, UN...",37215.0,RED RIVER,3303,34.0,1728273.001,639424.0007,-86.813033,36.087218
20119,266165,10/30/2019,Trash Collection Complaint,Driver/ Trash guys dropped the bin along with ...,"617 MERRYLODGE CT, 37207",37207.0,RED RIVER,2201,2.0,1741620.073,688279.7960,-86.769162,36.221716
20121,266174,10/30/2019,Trash Collection Complaint,Trash dumped and left in alley on pickup day,"6106B CALIFORNIA AVE, NASHVILLE, TN 37209, USA",37209.0,METRO,9302,20.0,1714370.132,668325.7122,-86.860944,36.166282
20124,266207,10/30/2019,Trash Collection Complaint,The last 3 weeks our trash has been scattered ...,"6109A NEW YORK AVE, NASHVILLE, TN 37209, USA",37209.0,METRO,9302,20.0,1714479.448,668534.3564,-86.860580,36.166857
20127,266263,10/31/2019,Trash Collection Complaint,Caller stated she is an elderly. Trash guys di...,"910 JESSAMIN RD, NASHVILLE, TN 37204, UNITED S...",37204.0,METRO,9406,17.0,1741427.079,655324.7905,-86.768938,36.131188


> <div class="alert alert-block alert-info">
    > A: placement of trash carts after pick-up, driving complaints, too early pick-ups, spilt trash

## *Q: Are there any geospatial analysis you can do?  Which visualizations can you create? **to do* 

## Q: How do metro crews compare to the contractor's performance?

In [184]:
trash['Trash Hauler'].unique()

array(['RED RIVER', 'WASTE IND', 'METRO', nan], dtype=object)

In [185]:
trash['Request'].unique()

array(['Trash - Backdoor', 'Trash - Curbside/Alley Missed Pickup',
       'Trash Collection Complaint', 'Damage to Property'], dtype=object)

In [186]:
trash['Year'] = pd.DatetimeIndex(trash['Date Opened']).year
trash.head()

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y,LONGITUDE,LATITUDE,Year
0,25270,11/1/2017,Trash - Backdoor,"house with the wheel chair ramp, they share dr...",3817 CROUCH DR,37207.0,RED RIVER,3205,2.0,1727970.412,686779.4781,-86.815392,36.217292,2017
1,25274,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 CLARKSVILLE PIKE,37218.0,RED RIVER,4202,1.0,1721259.366,685444.7996,-86.838103,36.213470,2017
2,25276,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 THUNDERBIRD DR,37209.0,RED RIVER,4205,20.0,1707026.753,659887.4716,-86.885562,36.142923,2017
3,25307,11/1/2017,Trash - Curbside/Alley Missed Pickup,missed,2603 OLD MATTHEWS RD,37207.0,WASTE IND,2206,2.0,1735691.771,685027.2459,-86.789170,36.212652,2017
4,25312,11/1/2017,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 CROLEY DR,37209.0,RED RIVER,4203,20.0,1710185.772,664205.1011,-86.874995,36.154861,2017


In [189]:
missed_pickup = trash['Request'] == 'Trash - Curbside/Alley Missed Pickup'
red_river = trash['Trash Hauler'] == 'RED RIVER'
trash_missed_red_river = trash[missed_pickup & red_river]
missed_red_river = trash_missed_red_river[['Year', 'Request']].groupby('Year').count()
missed_red_river = missed_red_river.rename(columns={'Request': 'RR Missed'}) 
missed_red_river.loc['Total']= missed_red_river.sum()
display(missed_red_river)

,RR Missed
Year,
2017,429
2018,3444
2019,6842
Total,10715


In [190]:
complaint = trash['Request'] == 'Trash Collection Complaint'
red_river = trash['Trash Hauler'] == 'RED RIVER'
complaint_red_river = trash[complaint & red_river]
complaint_red_river = complaint_red_river[['Year', 'Request']].groupby('Year').count()
complaint_red_river = complaint_red_river.rename(columns={'Request': 'RR Complaint'}) 
complaint_red_river.loc['Total']= complaint_red_river.sum()
display(complaint_red_river)

,RR Complaint
Year,
2017,122
2018,436
2019,729
Total,1287


In [191]:
damage = trash['Request'] == 'Damage to Property'
red_river = trash['Trash Hauler'] == 'RED RIVER'
damage_red_river = trash[damage & red_river]
damage_red_river = damage_red_river[['Year', 'Request']].groupby('Year').count()
damage_red_river = damage_red_river.rename(columns={'Request': 'RR Damage'}) 
damage_red_river.loc['Total']= damage_red_river.sum()
display(damage_red_river)

,RR Damage
Year,
2017,17
2018,80
2019,100
Total,197


In [192]:
missed_pickup = trash['Request'] == 'Trash - Curbside/Alley Missed Pickup'
metro = trash['Trash Hauler'] == 'METRO'
trash_missed_metro = trash[missed_pickup & metro]
missed_metro = trash_missed_metro[['Year', 'Request']].groupby('Year').count()
missed_metro = missed_metro.rename(columns={'Request': 'M Missed'}) 
missed_metro.loc['Total']= missed_metro.sum()
display(missed_metro)

,M Missed
Year,
2017,122
2018,1374
2019,1192
Total,2688


In [193]:
complaint = trash['Request'] == 'Trash Collection Complaint'
metro = trash['Trash Hauler'] == 'METRO'
complaint_metro = trash[complaint & metro]
complaint_metro = complaint_metro[['Year', 'Request']].groupby('Year').count()
complaint_metro = complaint_metro.rename(columns={'Request': 'M Complaint'}) 
complaint_metro.loc['Total']= complaint_metro.sum()
display(complaint_metro)

,M Complaint
Year,
2017,45
2018,312
2019,261
Total,618


In [194]:
damage = trash['Request'] == 'Damage to Property'
metro = trash['Trash Hauler'] == 'METRO'
damage_metro = trash[damage & metro]
damage_metro = damage_metro[['Year', 'Request']].groupby('Year').count()
damage_metro = damage_metro.rename(columns={'Request': 'M Damage'}) 
damage_metro.loc['Total']= damage_metro.sum()
display(damage_metro)

,M Damage
Year,
2017,2
2018,21
2019,10
Total,33


In [203]:
comparison_table = missed_red_river.join([missed_metro, complaint_red_river, complaint_metro, damage_red_river, damage_metro])
comparison_table['RR Total %'] = round((comparison_table['RR Missed'] + comparison_table['RR Complaint'] + comparison_table['RR Damage']) / (comparison_table.sum(axis=1)), 2)
comparison_table['M Total %'] = round((comparison_table['M Missed'] + comparison_table['M Complaint'] + comparison_table['M Damage']) / (comparison_table.sum(axis=1)), 2)
display(comparison_table)

,RR Missed,M Missed,RR Complaint,M Complaint,RR Damage,M Damage,RR Total %,M Total %
Year,,,,,,,,
2017,429,122,122,45,17,2,0.77,0.23
2018,3444,1374,436,312,80,21,0.70,0.30
2019,6842,1192,729,261,100,10,0.84,0.16
Total,10715,2688,1287,618,197,33,0.79,0.21


> <div class="alert alert-block alert-info">
    > A: Red River receives a significantly higher number of requests. However, how this compares to the total number of pick-ups they each complete yearly is unknown, so a true comparison is difficult.

## Q: How much does each trash hauler owe?

In [204]:
trash['Trash Hauler'].unique()

array(['RED RIVER', 'WASTE IND', 'METRO', nan], dtype=object)

In [217]:
missed_pickup = trash['Request'] == 'Trash - Curbside/Alley Missed Pickup'
red_river = trash['Trash Hauler'] == 'RED RIVER'
waste_ind = trash['Trash Hauler'] == 'WASTE IND'
metro = trash['Trash Hauler'] == 'METRO'
nan = trash['Trash Hauler'] == 'nan'

In [214]:
red_river_missed = trash[missed_pickup & red_river]
duplicates = red_river_missed.duplicated(subset = ['Incident Address'], keep = 'first')
red_river_missed_charged = red_river_missed[duplicates]
red_river_fines = red_river_missed_charged['Incident Address'].count() * 200
print(red_river_fines)

528200


In [215]:
waste_ind_missed = trash[missed_pickup & waste_ind]
duplicates = waste_ind_missed.duplicated(subset = ['Incident Address'], keep = 'first')
waste_ind_missed_charged = waste_ind_missed[duplicates]
waste_ind_fines = waste_ind_missed_charged['Incident Address'].count() * 200
print(waste_ind_fines)

46600


In [216]:
metro_missed = trash[missed_pickup & metro]
duplicates = metro_missed.duplicated(subset = ['Incident Address'], keep = 'first')
metro_missed_charged = metro_missed[duplicates]
metro_fines = metro_missed_charged['Incident Address'].count() * 200
print(metro_fines)

128800


In [218]:
nan_missed = trash[missed_pickup & nan]
duplicates = nan_missed.duplicated(subset = ['Incident Address'], keep = 'first')
nan_missed_charged = nan_missed[duplicates]
nan_fines = nan_missed_charged['Incident Address'].count() * 200
print(nan_fines)

0


> <div class="alert alert-block alert-info">
    A: Red River owes \$528,200, Waste Ind owes \$46,600 and Metro owes \$128,800

## Q: What were to total missed pickup by route?

In [231]:
missed_pickup = trash['Request'] == 'Trash - Curbside/Alley Missed Pickup'
trash_missed = trash[missed_pickup]
missed_route = trash_missed.groupby('Trash Route').count()
missed_route = missed_route[['Request Number']]
missed_route_sorted = missed_route.sort_values(by=['Request Number'], ascending=False)
missed_route_sorted.loc['Total']= missed_route_sorted.sum()
display(missed_route_sorted)

,Request Number
Trash Route,
4504,305
4404,246
3302,230
3305,215
9303,209
...,...
4201S,2
3303S,2
2505S,2


> <div class="alert alert-block alert-info">
A: See chart above